# Torchmetrics 패키지
- pytorch에서 모델 성능평가 기능 제공 패키지
- 추가 설치 필요 
    - pip : pip install torchmetrics 
    - conda : conda install -c conda-forge torchmetrics 
- 사용법 
    - 클래스 방식 : 인스턴스 생성(Heap에 생성) 후 사용  
        - 정적 클래스 : 인스턴스 생성 x 
    - 함수 방식 : 바로 사용 

## [1] 회귀 (Regression) 성능 지표

In [16]:
# 모듈 로딩 
import torch
from torchmetrics.functional.regression import r2_score 
from torchmetrics.regression import R2Score, MeanSquaredError, MeanAbsoluteError


In [17]:
# 클래스 방식 ------------------------------------------
target = torch.tensor([3, -0.5, 2, 7]) # 1D (4,)
preds = torch.tensor([2.5, 0.0, 2, 8]) # 1D (4,)

# 성능 평가 => R2Score : 0 ~ 1 (좋음) 정답에 얼마나 근사하게 표현했는가
r2score = R2Score()
r2score(preds, target)

tensor(0.9486)

In [18]:
def add(a,b): return a+b

In [19]:
my_func = add 
# my_func이 add 주소 가지고 있음 -> 메모리 상에서 공유 -> 변수명으로 함수 사용 ㅇ 
my_func(10,20) 
# => 재사용 가능 

30

In [20]:
target = torch.tensor([3, -0.5, 2, 7]) # 1D (4,)
preds = torch.tensor([2.5, 0.0, 2, 8]) # 1D (4,)

# 클래스 사용해서 함수로 사용 ㅇ 
# 클래스 -> 메모리에 인스턴스 생성 
R2Score()(preds, target) # 한 번만 쓰고 끝남 => 재사용 불가 
# R2Score주소(데이터1, 데이터2) -> R2Score 인스턴스로 향함 

tensor(0.9486)

In [21]:
# 함수 방식 ---------------------------------------------------------
target = torch.tensor([3, -0.5, 2, 7]) # 1D (4,)
preds = torch.tensor([2.5, 0.0, 2, 8]) # 1D (4,)

# 성능 평가 => R2Score : 0 ~ 1 (좋음) 정답에 얼마나 근사하게 표현했는가
r2score(preds, target)

tensor(0.9486)

In [22]:
class MyR2(R2Score):
    pass

In [23]:
gg = MyR2()
gg(preds, target)

tensor(0.9486)

In [26]:
target = torch.tensor([2.5, 5.0, 4.0, 8.0])
preds = torch.tensor([3.0, 5.0, 2.5, 7.0])

# 오차 계산 : 0에 가까울 수록 좋은 것 
mean_squared_error = MeanSquaredError()
print(mean_squared_error(preds, target)) # 함수 사용 
print(MeanSquaredError()(preds, target)) # 클래스 사용 

tensor(0.8750)
tensor(0.8750)


### 분류(Classfication) 성능 평가 지표 
- Accuracy : 정확도 (양성 - 양성, 음성 - 음성) => 불균형 데이터일 경우 신뢰성 없음 
- Precision (예측 양성 -> 실제 양성)
- Recall (실제 양성 -> 예측 양성)
- F1-Score
- ConfusionMatrix
- ROC-AUC => 의료분야 / 2진 분류
    

In [27]:
from torchmetrics.classification import F1Score, Accuracy, Precision, Recall
from torchmetrics.functional.classification import f1_score, accuracy, precision, recall

In [31]:
# 2진 분류 => task = "binary"

from torchmetrics.classification import BinaryF1Score

target = torch.tensor([0, 1, 1, 0, 1, 0])
preds = torch.tensor([0, 1, 1, 0, 0, 1])
f1 = F1Score(task="binary")
print(f1(preds, target))
print(BinaryF1Score()(preds, target))

tensor(0.6667)
tensor(0.6667)


In [36]:
# 다중 분류 => task = "multiclass"

from torchmetrics.classification import MulticlassF1Score

target = torch.tensor([0, 1, 2, 0, 2, 0])
preds = torch.tensor([0, 2, 1, 0, 2, 1])
f1 = F1Score(task="multiclass", num_classes=3)

print(f1(preds, target)) 
print(MulticlassF1Score(num_classes=3)(preds, target)) 

# => 평균 내는 방법이 다름 

tensor(0.5000)
tensor(0.4333)


In [37]:
# 다중 분류 => task = "multiclass"

from torchmetrics.classification import MulticlassF1Score

target = torch.tensor([0, 1, 2, 0, 2, 0])
preds = torch.tensor([0, 2, 1, 0, 2, 1])

# F1Score => average = 'micro' (sum statistics over all labels)
f1 = F1Score(task="multiclass", num_classes=3, average='macro')  
# num_classes : 변환된 범주형 데이터에서 클래스 수를 결정

print(f1(preds, target)) 

# MulticlassF1Score => average = 'macro' (calculate statistics for each label and average them)
print(MulticlassF1Score(num_classes=3)(preds, target)) 


tensor(0.4333)
tensor(0.4333)


-> 평균내는 방식을 맞추면 값이 동일해진다 

In [40]:
# 2진 분류 => task = "binary"

from torchmetrics.classification import BinaryPrecision

target = torch.tensor([0, 1, 1, 0, 1, 0])
preds = torch.tensor([0, 1, 1, 0, 0, 1])

pb = Precision(task='binary', threshold=0.7)
print(pb(preds, target))

print(BinaryPrecision(threshold=0.7)(preds, target))

tensor(0.6667)
tensor(0.6667)


In [42]:
# 다중 분류 => task = "multiclass"

from torchmetrics.classification import MulticlassPrecision

target = torch.tensor([0, 1, 2, 0, 2, 0])
preds = torch.tensor([0, 2, 1, 0, 2, 1])

# F1Score => average = 'micro' (sum statistics over all labels)
# f1 = Precision(task="multiclass", num_classes=3, average='macro')
# average = 'macro' 를 안 넣어도 값 동일 => 데이터가 동일하면 값 동일 
f1 = Precision(task="multiclass", num_classes=3)

print(f1(preds, target)) 

# MulticlassF1Score => average = 'macro' (calculate statistics for each label and average them)
print(MulticlassPrecision(num_classes=3)(preds, target)) 



tensor(0.5000)
tensor(0.5000)
